%%markdown

# GigaSpaces Aggregation Examples
This notebook will demonstrate the basics of the GigaSpaces aggregation API.



%%markdown

##### GigaSpaces start up instructions

1. In a console window, go to the GigaSpaces install directory/bin (for convenience GS_HOME has already been defined).

`cd $GS_HOME/bin`

2. Start the service grid with a manager and 4 GSCs (in developer mode)

`./gs.sh host run-agent --auto --gsc=4`

3. In another console window, deploy a space named demo.

`./gs.sh space deploy --partitions=2 --ha demo`


%%markdown

##### POJO definition

1. The accompanying Maven project can be found at ~/gs-core-dev-training/jupyter. `mvn install` has been run and it is now accessible to this notebook.

In [ ]:
%%pom
repositories:
  - id: org.openspaces
    url: https://maven-repository.openspaces.org
dependencies:
  - groupId: org.gigaspaces
    artifactId: xap-openspaces
    version: 16.4.0

  - groupId: com.gigaspaces.training
    artifactId: model
    version: 1.0
        

In [ ]:
%%classpath

In [ ]:
%%java

import com.gigaspaces.training.model.Product;

import com.j_spaces.core.client.SQLQuery;

import org.openspaces.core.GigaSpace;
import org.openspaces.core.GigaSpaceConfigurer;
import org.openspaces.core.space.EmbeddedSpaceConfigurer;

import com.gigaspaces.query.aggregators.AggregationResult;
import com.gigaspaces.query.aggregators.AggregationSet;

import java.math.BigDecimal;



In [ ]:
%%java
// an example of instantiating an embedded space.

EmbeddedSpaceConfigurer configurer = new EmbeddedSpaceConfigurer("demo");
GigaSpace gigaSpace = new GigaSpaceConfigurer(configurer).gigaSpace();


In [ ]:
%%java

// write objects
String[] names = new String[]{ "Bicyle", "Playing Cards", "Chess Set", "Football", "Pickleball Paddle", 
        "Fishing Rod", "Soccer Ball", "Golf Club Set", "Frisbee", "Table Tennis Paddle",
        "Cooler", "Water Bottle", "Baseball Glove", "Hockey Stick", "Hockey Puck" };
    
    
for( int i=0; i < 15; i++) {
    Product product = new Product();
    product.setId(i);
    product.setName(names[i]);
    product.setPrice(BigDecimal.valueOf(i + 1));
    gigaSpace.write(product);
}

In [ ]:
%%java
// count

int numObjects = gigaSpace.count(new Product());
System.out.println("The number of Product objects is: " + numObjects);

In [ ]:
%%java

// aggregation example
SQLQuery<Product> sqlQuery = new SQLQuery<Product>(Product.class, "");

AggregationResult aggregationResult = gigaSpace.aggregate(sqlQuery,
        new AggregationSet()
            .maxEntry("price").minEntry("price")
            .sum("price").average("price")
            .minValue("price").maxValue("price"));

Product highestPricedProduct  = (Product) aggregationResult.get(0);
Product lowestPricedProduct = (Product) aggregationResult.get(1);
BigDecimal totalPrice = (BigDecimal) aggregationResult.get(2);
BigDecimal averagePrice = (BigDecimal) aggregationResult.get(3);
BigDecimal minPrice = (BigDecimal) aggregationResult.get(4);
BigDecimal maxPrice = (BigDecimal) aggregationResult.get(5);

System.out.println("highestPricedProduct is: " + highestPricedProduct);
System.out.println("lowestPricedProduct is: " + lowestPricedProduct);
System.out.println("totalPrice is: " + totalPrice);
System.out.println("averagePrice is: " + averagePrice);
System.out.println("minPrice is: " + minPrice);
System.out.println("maxPrice is: " + maxPrice);

